In [4]:
# import necessary libraries
import time 
import random
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')


In [6]:

url = 'https://www.imdb.com/'
path = 'D:/chromedriver-win64/chromedriver.exe'

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection

s = Service(path)

driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get(url)

driver.find_element(By.XPATH, '''//*[@id="suggestion-search-button"]''').click()
time.sleep(2)

element = driver.find_element(By.XPATH, '''//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/div[2]/a[1]''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()

time.sleep(2)
element = driver.find_element(By.XPATH, '''//*[@id="accordion-item-titleTypeAccordion"]/div/section/button[1]''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()

element = driver.find_element(By.XPATH, '''//*[@id="releaseDateAccordion"]/div[1]/label''')
actions = ActionChains(driver)
actions.move_to_element(element).perform()
time.sleep(1) 
element.click()
time.sleep(2)

search1 = driver.find_element(By.CSS_SELECTOR, 'input[class="ipc-textinput__input"][name="release-yearmonth-start-input"]')
actions = ActionChains(driver)
actions.move_to_element(search1).perform()
search1.click()
search1.send_keys(2014)

time.sleep(2)
search2 = driver.find_element(By.CSS_SELECTOR, 'input[class="ipc-textinput__input"][name="release-yearmonth-end-input"]')
search2.click()
search2.send_keys(2024)

driver.find_element(By.XPATH, '''//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[1]/button''').click()
time.sleep(4)

# here i have click the more button for upto 10,000 movies
for click in range(200):
    try:       
        element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button')
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        time.sleep(random.uniform(2, 3)) 
        element.click()
        time.sleep(random.uniform(2, 4))
    except Exception as e:
        print('No more movies. Exist the loop', e)
        break
        
time.sleep(random.uniform(1,2))
content = driver.page_source

print('Scrolling stoppped at:', datetime.datetime.now())

Scrolling stoppped at: 2025-01-31 19:01:21.936084


In [13]:
# here i have gone through each movie, extracted the necessary data to impute our original dataset
# and appended them on a list
soup = BeautifulSoup(content, 'lxml')

containers = soup.find_all( 'div', class_="ipc-metadata-list-summary-item__tc")

details = [['Movie Name', 'Release Year', 'Duration', 'Ratings', 'Number_of_Ratings', 'Rated', 
            'Metascore', 'Description', 'Movie Image']]
for i in containers:
    try:
        name = i.find('h3',class_="ipc-title__text").text
    except:
        name = np.nan

    
    three_items = i.find_all('span', class_="sc-ad5a2436-7 cJVQtZ dli-title-metadata-item")
    try:
        release_yr = three_items[0].text
    except:
        release_yr = np.nan

    try:
        duration = three_items[1].text
    except:
        duration = np.nan

    try:
        rated = three_items[2].text
    except:
        rated = np.nan

    try:
        ratings = i.find('span',class_="ipc-rating-star--rating").text
    except:
        ratings = np.nan

    try:
        no_of_ratings = i.find('span',class_="ipc-rating-star--voteCount").text
    except:
        no_of_ratings = np.nan
        
    try:
        meta_score  = i.find('span', class_="sc-b0901df4-0 bXIOoL metacritic-score-box").text
    except:
        meta_score = np.nan

    try:
        description = i.find('div', class_="ipc-html-content-inner-div").text
    except:
        description = np.nan
        
    try:
        photos = i.find('img', class_="ipc-image")
        photos = photos['src']
    except:
        photos = np.nan

    details.append([name, release_yr, duration, ratings, no_of_ratings, rated, meta_score, description, photos])

# created a dataframe of the extracted movies data to impute original dataset
df = pd.DataFrame(details[1:], columns=details[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Movie Name         10000 non-null  object
 1   Release Year       10000 non-null  object
 2   Duration           9875 non-null   object
 3   Ratings            9970 non-null   object
 4   Number_of_Ratings  9970 non-null   object
 5   Rated              6402 non-null   object
 6   Metascore          4921 non-null   object
 7   Description        9977 non-null   object
 8   Movie Image        9996 non-null   object
dtypes: object(9)
memory usage: 703.3+ KB


In [14]:
df

,Movie Name,Release Year,Duration,Ratings,Number_of_Ratings,Rated,Metascore,Description,Movie Image
0,1. Nosferatu,2024,2h 12m,7.4,(124K),R,78,A gothic tale of obsession between a haunted y...,https://m.media-amazon.com/images/M/MV5BY2FhZG...
1,2. The Brutalist,2024,3h 34m,8.0,(14K),R,90,When a visionary architect and his wife flee p...,https://m.media-amazon.com/images/M/MV5BM2U0MW...
2,3. The Substance,2024,2h 21m,7.3,(224K),R,78,A fading celebrity takes a black-market drug: ...,https://m.media-amazon.com/images/M/MV5BZDQ1NG...
3,4. Emilia Pérez,2024,2h 12m,5.9,(49K),R,71,Emilia Pérez follows four remarkable women in ...,https://m.media-amazon.com/images/M/MV5BNjBhMW...
4,5. Gladiator II,2024,2h 28m,6.6,(185K),R,64,After his home is conquered by the tyrannical ...,https://m.media-amazon.com/images/M/MV5BMWYzZT...
...,...,...,...,...,...,...,...,...,...
9995,9996. Leanne Morgan: I'm Every Woman,2023,1h 14m,7.2,(668),NaN,NaN,Morgan showcases her life via intimate tales a...,https://m.media-amazon.com/images/M/MV5BNmVkMz...
9996,9997. Mafia Inc,2019,2h 23m,6.8,(1.7K),NaN,64,"The Gamache, tailors from father to son, have ...",https://m.media-amazon.com/images/M/MV5BNTRkZj...
9997,9998. Pound of Flesh,2015,1h 44m,5.1,(7.2K),R,41,A man's heroic attempt to help a woman in dist...,https://m.media-amazon.com/images/M/MV5BMjI2OD...
9998,9999. The One I Wrote for You,2014,1h 50m,6.0,(445),PG,NaN,"A singer and songwriter, who gave up his dream...",https://m.media-amazon.com/images/M/MV5BMGY5MT...


In [161]:

def rating_in_num(col):
    col = str(col)
    if 'K' in col:
        return float(col.replace('K', '')) * 1000     
    elif 'M' in col:
        return float(col.replace('M', '')) * 1000000
    else:    
        return float(col)
        

In [168]:
# here i did a little bit cleaning so it matches the original dataset format
df['Movie Name'] = df['Movie Name'].str.replace(r'^\d+\.', '', regex =True)
# df['Number_of_Ratings'].str.replace(r'\(|\)', '', regex=True)'\xa02.3M'
df['Number_of_Ratings'] = df['Number_of_Ratings'].str.replace(r'[xa()]', '', regex=True).apply(rating_in_num)


In [169]:
df

,Movie Name,Release Year,Duration,Ratings,Number_of_Ratings,Rated,Metascore,Description,Movie Image
0,Nosferatu,2024,2h 12m,7.4,124000.0,R,78,A gothic tale of obsession between a haunted y...,https://m.media-amazon.com/images/M/MV5BY2FhZG...
1,The Brutalist,2024,3h 34m,8.0,14000.0,R,90,When a visionary architect and his wife flee p...,https://m.media-amazon.com/images/M/MV5BM2U0MW...
2,The Substance,2024,2h 21m,7.3,224000.0,R,78,A fading celebrity takes a black-market drug: ...,https://m.media-amazon.com/images/M/MV5BZDQ1NG...
3,Emilia Pérez,2024,2h 12m,5.9,49000.0,R,71,Emilia Pérez follows four remarkable women in ...,https://m.media-amazon.com/images/M/MV5BNjBhMW...
4,Gladiator II,2024,2h 28m,6.6,185000.0,R,64,After his home is conquered by the tyrannical ...,https://m.media-amazon.com/images/M/MV5BMWYzZT...
...,...,...,...,...,...,...,...,...,...
9995,Leanne Morgan: I'm Every Woman,2023,1h 14m,7.2,668.0,NaN,NaN,Morgan showcases her life via intimate tales a...,https://m.media-amazon.com/images/M/MV5BNmVkMz...
9996,Mafia Inc,2019,2h 23m,6.8,1700.0,NaN,64,"The Gamache, tailors from father to son, have ...",https://m.media-amazon.com/images/M/MV5BNTRkZj...
9997,Pound of Flesh,2015,1h 44m,5.1,7200.0,R,41,A man's heroic attempt to help a woman in dist...,https://m.media-amazon.com/images/M/MV5BMjI2OD...
9998,The One I Wrote for You,2014,1h 50m,6.0,445.0,PG,NaN,"A singer and songwriter, who gave up his dream...",https://m.media-amazon.com/images/M/MV5BMGY5MT...


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Movie Name         10000 non-null  object 
 1   Release Year       10000 non-null  object 
 2   Duration           9875 non-null   object 
 3   Ratings            9970 non-null   object 
 4   Number_of_Ratings  9970 non-null   float64
 5   Rated              6402 non-null   object 
 6   Metascore          4921 non-null   object 
 7   Description        9977 non-null   object 
 8   Movie Image        9996 non-null   object 
dtypes: float64(1), object(8)
memory usage: 703.3+ KB


In [171]:
# saved the dataframe in a csv file so we can impute it in our original dataset missing values
df.to_csv('imputation_imdb.csv', index = False)